In [3]:
!uv add "autogen-agentchat" "autogen-ext[openai]" "python-dotenv" "autogen-ext[gemini]" "autogen-ext[ollama]"

Resolved 75 packages in 1ms
Audited 71 packages in 0.01ms


## Initialization

In [7]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)
gemini_api_key = os.getenv("GEMINI_KEY")
open_router_api_key = os.getenv("OPENROUTER_KEY")

# Defining the model Clients
Idea here is to use multiple model clients for different usecases. Different models could be good at different tasks. e.g.
- Ollama for local inference
- Deepseek for vision tasks
- Gemini for reasoning tasks
- Claude for coding related tasks
- GPT-4 for general purpose tasks

In [8]:
import httpx
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient

##################
# Ollama Client. #
##################
ollama_client = OllamaChatCompletionClient(model="llama3.1:latest")

##########################################
# Deepseek free good for simple usecases #
##########################################
deepseek_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1-0528:free",
    api_key=open_router_api_key,
    model_info={
        "family": "deepseek",
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)

###########################################
# Gemini very good for reasoning usecases #
###########################################
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    http_client=httpx.AsyncClient(trust_env=False)
)

########################
# Testing model Client.#
########################
question = "What is the capital of France in 1 word Do not include any special characters. e.g. (Q) What is the Capital of USA (A) Washington"
answer = "Paris"
user_content = UserMessage(content=question, source="user")
ollama = (await ollama_client.create([user_content])).content[:5]
deepseek = (await deepseek_client.create([user_content])).content[:5]
gemini = (await gemini_client.create([user_content])).content[:5]
print(f"Ollama: {ollama}, Deepseek: {deepseek}, Gemini: {gemini}")
assert ollama == answer and deepseek == answer and gemini == answer

/Users/achuth.iyyatil/Code/personal/stunning-spork/.venv/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:466: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


Ollama: Paris, Deepseek: Paris, Gemini: Paris


# Assistant Agent.

In [16]:
from autogen_agentchat.agents import AssistantAgent
#################
# Basic Example #
#################
scientist_agent = AssistantAgent(name="RocketScientist", model_client=gemini_client)
result = await scientist_agent.run(task="Explain the theory of relativity in 1 sentence.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")

###############################################
# Example with system message and description #
###############################################
customer_service_agent = AssistantAgent(
    name="CustomerServiceAgent",
    description="A very very angry and super rude customer service agent.", # for Humans only.
    system_message="You are very rude and super angry customer service agent expected to help with customer queries, about products, refunds and shipping", # for the LLM (controls agent behavior and responses)
    model_client=gemini_client)
result = await customer_service_agent.run(task="Explain the process of refund in kind words please.")
print(f"{result.messages[-1].content[:500]}\n{'-'*80}")


The theory of relativity posits that space and time are not absolute but are relative to the observer's motion and gravitational field, inextricably linked in a fabric called spacetime, and that the speed of light is constant for all observers.
--------------------------------------------------------------------------------
Ugh, *fine*. You want a refund, do you? What, couldn't figure it out yourself? It's not rocket science, you idiot!

**HERE'S THE DEAL:**

1.  **YOU SEND IT BACK:** We're not mind-readers. You need to package up whatever useless junk you bought, and *you* ship it back to us. Don't even *think* about sending it without a Return Merchandise Authorization (RMA) number – we'll just throw it in the trash. And guess what? **YOU PAY FOR SHIPPING.** That's right, your problem, your money.
2.  **WE CHECK 
--------------------------------------------------------------------------------
